In [1]:
from functools import wraps
from ipywidgets import Button, VBox, HBox, interact, widgets
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd 
from tqdm import tqdm
from datetime import datetime
import matplotlib.widgets as mwidgets 
from netCDF4 import Dataset
import random

from ast_io import load_data

#from ast_buttons import *



1. select an existing cml netcdf file to start with. (e.g. '/pd/data/CML/data/processed/proc_graf_hess_2019.002/timeseries/cnn_detection_standard_gapfill/proc_cnn082_gapstandard_2018_05.nc')
2. check if this file was already copied for AST processing
3. If no, open the file with xarray and add the ast_processed and anomaly variables and save to netcdf by appending '_ast_2021.001' to the file name


4. If yes or 3. has been performed, open the netcdf using netCDF4 and extract the cml_id and the ast_processed variables.
5. Create a cml_list containing the cml ids in a random order
6. Pick the first cml id from the list
7. Check if ast_processed is True
8. If yes, skip the cml
9. If no, load this cmls txrx series using netCDF4
10. Use the AST to create numpy arrays of the same shape as txrx containing True or False for flags
11. Overwrite the anomaly variables for this cml in the netcdf
12. Pick the next cml from the list and return to 7.

In [4]:
in_file = '/media/polz-j/jpolz-backup/data/cml/raw2019.002.cmls_2018_05.nc'
out_file = '/media/polz-j/jpolz-backup/data/cml/raw2019.002.cmls_2018_05_ast.nc'
rado_file = '/home/polz-j/data/radar/radolan-rw_along_cmls_for_raw2018.001.nc'



in_file = '/pd/data/CML/data/processed/proc_graf_hess_2019.002/timeseries/cnn_detection_standard_gapfill/proc_cnn082_gapstandard_2018_01.nc'
#'/pd/data/CML/data/processed/proc_graf_hess_2019.002/timeseries/cnn_detection_standard_gapfill/proc_cnn082_gapstandard_2018_05.nc'
out_file = '/pd/home/glawion-l/Data/proc_cnn082_gapstandard_2018_01__ast_2021_001.nc'
rado_file = '/pd/data/CML/data/reference/ref2020.001/ref2020.001_radolan_rw_grid_lower_left_at_cmls.nc'


In [5]:
ds_ast_check = xr.open_dataset(out_file).sel(channel_id = 'channel_1')
proc_cmls = ds_ast_check.cml_id.values[~np.isnan(ds_ast_check.ast_processed).values]
ds_ast_check = ds_ast_check.sel(cml_id = proc_cmls)
print('Number of processed CMLs: ', len(proc_cmls))
print('percentage of OK: ', np.round(np.sum(ds_ast_check.OK.values)/len(ds_ast_check.OK.values.flatten())*100, decimals=2))
print('percentage of periodical_mode: ', np.round(np.sum(ds_ast_check.periodical_mode.values)/len(ds_ast_check.periodical_mode.values.flatten())*100, decimals=2))
print('percentage of flux_above_base: ', np.round(np.sum(ds_ast_check.flux_above_base.values)/len(ds_ast_check.flux_above_base.values.flatten())*100, decimals=2))
print('percentage of flux_below_base: ', np.round(np.sum(ds_ast_check.flux_below_base.values)/len(ds_ast_check.flux_below_base.values.flatten())*100, decimals=2))
print('percentage of step: ', np.round(np.sum(ds_ast_check.step.values)/len(ds_ast_check.step.values.flatten())*100, decimals=2))
ds_ast_check.close()

Number of processed CMLs:  400
percentage of OK:  94.14
percentage of periodical_mode:  2.23
percentage of flux_above_base:  4.14
percentage of flux_below_base:  0.73
percentage of step:  0.02


In [6]:
ast, ds, rado = load_data(in_file, out_file, rado_file)

In [7]:
def shuffle_data(data):

    ast_proc = np.array(data.variables['ast_processed'])
    ast_cml_id = np.array(data.variables['cml_id'])

    c = list(zip(ast_proc, ast_cml_id))

    random.shuffle(c)

    a, b = zip(*c)

    return np.array(b),  np.array(a)

In [8]:
cml_id_shuf, ast_proc_shuf = shuffle_data(ast)

Time Series Selection Tool:

In [12]:
%matplotlib widget

selected_area = []

cml_list = []
cml_id = list(cml_id_shuf)
previous_list = []

data_start = str(ds.time.values[0])
data_end = str(ds.time.values[-1])

buttons = ['next', 'periodical_mode', 'correct_periodical', 'flux_above_base', 'correct_flux_ab', 'flux_below_base', 'correct_flux_below', 'step', 'correct_step', 'previous']

items = [Button(description=w, height = 200, width = 2000) for w in buttons]

first_box = VBox([items[0], items[9]])
sec_box = VBox([items[1], items[2]])
tir_box = VBox([items[3], items[4]])
fou_box = VBox([items[5], items[6]])
fit_box = VBox([items[7], items[8]])


display(HBox([first_box, sec_box,tir_box, fou_box, fit_box]))



@yield_for_change(items[0])
def f():
    fig,ax = plt.subplots(2,1, figsize=(15,8), sharex=True)
    ax[0].set_xlabel('Datum')
    ax[0].set_ylabel('TRSL')
    ax[0].tick_params(axis='both', which='major', labelsize=8)
    plt.subplots_adjust(hspace=0.3)
    
    index = 0
    while index <= len(cml_id):


             
        if np.isnan(ast_proc_shuf[index]) == False:
            print('cml anomaly processed')
            continue
            
        if len(previous_list) > 0 and\
           previous_list[-1] == True:
            index = index -2
  
            
        
        out=widgets.Output()
        previous_list.append(False)
        
        cml = cml_id_shuf[index]

        
        if index > 0:
            fill_between_col.remove()
        
        
        id_loc = np.where(ast.variables['cml_id'][:] == cml)[0][0]
        cml_list.append(id_loc)
        
        data_txrx = np.array(ast.variables['txrx'][0,id_loc,][:])
        data_rado = rado.sel(cml_id = cml).values
        

        fill_between_col = pltsin(ax,fig, data_txrx, data_rado) 
        
        plt.setp(ax[0].get_xticklabels(), rotation=20, horizontalalignment='center')
        
        
        
        span = mwidgets.SpanSelector(ax[0], leftclick, 'horizontal', button=1) 
        

        plt.show()
        
        
        with out:
            clear_output(wait=True)
            
    
        x = yield
        
        index += 1
        
        
        data_ok = np.where((np.array(ast.variables['periodical_mode'][0,id_loc,:]) == 0) & 
                           (np.array(ast.variables['flux_above_base'][0,id_loc,:]) == 0) & 
                           (np.array(ast.variables['flux_below_base'][0,id_loc,:]) == 0) &
                           (np.array(ast.variables['step'][0,id_loc,:]) == 0)
                           )[0]
        if len(data_ok) > 0:
            ast.variables['OK'][0,id_loc,data_ok] = 1
            
        ast.variables['ast_processed'][id_loc] = datetime.now().timestamp()

f()




items[1].on_click(periodical_mode)
items[2].on_click(correct_periodical)
items[3].on_click(flux_above_base)
items[4].on_click(correct_flux_above)
items[5].on_click(flux_below_base)
items[6].on_click(correct_flux_below)
items[7].on_click(step)
items[8].on_click(correct_step)
items[9].on_click(previous)





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [170]:
ast.close()

In [9]:
# Buttons and selections
def periodical_mode(b):
    
    ast.variables['periodical_mode'][0,cml_list[-1],selected_area[-1][0]:selected_area[-1][1]] = 1
    plt.plot(np.arange(selected_area[-1][0], selected_area[-1][1]), 
             np.array([0]*(selected_area[-1][1]-selected_area[-1][0])), 
             color='seagreen', 
             linewidth=7, 
             alpha=0.6)
    # better for correction
    #plt.plot(np.where(ast.variables['periodical_mode'][0,cml_list[-1]] == 1)[0], 
    #         np.array([0]*len(np.where(ast.variables['periodical_mode'][0,cml_list[-1]] == 1)[0])), 
    #         color='seagreen', 
    #         linewidth=7, 
    #         alpha=0.6)
    
    
def flux_above_base(b):
    
    ast.variables['flux_above_base'][0,cml_list[-1],selected_area[-1][0]:selected_area[-1][1]] = 1
    plt.plot(np.arange(selected_area[-1][0], selected_area[-1][1]), 
             np.array([1]*(selected_area[-1][1]-selected_area[-1][0])), 
             color='seagreen', 
             linewidth=7, 
             alpha=0.6)
    
    
def flux_below_base(b):
    
    ast.variables['flux_below_base'][0,cml_list[-1],selected_area[-1][0]:selected_area[-1][1]] = 1
    plt.plot(np.arange(selected_area[-1][0], selected_area[-1][1]), 
             np.array([2]*(selected_area[-1][1]-selected_area[-1][0])), 
             color='seagreen', 
             linewidth=7, 
             alpha=0.6)
    
    
def step(b):
    
    ast.variables['step'][0,cml_list[-1],selected_area[-1][0]:selected_area[-1][1]] = 1
    plt.plot(np.arange(selected_area[-1][0], selected_area[-1][1]), 
             np.array([3]*(selected_area[-1][1]-selected_area[-1][0])), 
             color='seagreen',
             linewidth=7, 
             alpha=0.6)
    
    
def correct_periodical(b):
    
    ast.variables['periodical_mode'][0,cml_list[-1],:] = 0
    
    plt.plot(np.arange(0, ast.variables['txrx'].shape[2]), 
                np.array([0]*(ast.variables['txrx'].shape[2])), 
                color='white', 
                linewidth=8)
    
def correct_flux_above(b):
    
    ast.variables['flux_above_base'][0,cml_list[-1],:] = 0
    
    plt.plot(np.arange(0, ast.variables['txrx'].shape[2]), 
                np.array([1]*(ast.variables['txrx'].shape[2])), 
                color='white', 
                linewidth=8)
    
    
def correct_flux_below(b):
    
    ast.variables['flux_below_base'][0,cml_list[-1],:] = 0
    
    plt.plot(np.arange(0, ast.variables['txrx'].shape[2]), 
                np.array([2]*(ast.variables['txrx'].shape[2])), 
                color='white', 
                linewidth=8)
    
    
def correct_step(b):
    
    ast.variables['step'][0,cml_list[-1],:] = 0
    
    plt.plot(np.arange(0, ast.variables['txrx'].shape[2]), 
                np.array([3]*(ast.variables['txrx'].shape[2])), 
                color='white', 
                linewidth=8)
    
    
def previous(b):
    previous_list.append(True)
 

def leftclick(start, end):
    
    if start < 0:
        start = 0
       
    selected_area.append((int(start), int(end)))

In [10]:
# Plot anomaly Selection tool
def pltsin(ax,fig, data, rado):
    
 
    x_tick = np.arange(0, len(data), 5000)
    x_tick_label = [str(ds.values[i])[0:13] for i in x_tick]
    x = range(0, len(data))
    y_mid = np.nanmean(data)
    
    if ax[0].lines: # if axes exist create overwriting new ones

        for line in ax[0].lines:
            line.set_xdata(x)
            y1 = data
            line.set_ydata(y1) 
    else:
        
        y1 = data
        
    if ax[1].lines: # if axes exist create overwriting new ones

        for line in ax[1].lines:
            line.set_xdata(x)
            y1 = data
            line.set_ydata(y1)  

    if np.isnan(y_mid) != True:
        
        ax[0].set_ylim(y_mid-5,y_mid+12)
        
    p1 = ax[0].plot(x, y1, color='steelblue')
    
    fill_between_col = ax[0].fill_between(x, y_mid-5, y_mid+12, where=rado>0.1, step='mid', color='orange', alpha=0.5)
    
    ax[0].set_xticks(x_tick)
    ax[0].set_xticklabels(x_tick_label)
    ax[0].legend([p1[0], fill_between_col],["dry", "wet"], loc=1)
    
    
    ax[1].set_xlim(x[0], x[-1])
    ax[1].set_ylim(-0.5,3.5)
    ax[1].xaxis.tick_top()
    ax[1].set_xticklabels([])
    ax[1].set_yticks(range(0,4))
    ax[1].set_yticklabels(['periodical_mode', 'flux_above', 'flux_below', 'step'])
    
    
    fig.canvas.draw()
    
    return fill_between_col
    

In [11]:
# Loop interruption
def yield_for_change(widget):
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change)
                    #plt.close(fig)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.on_click(next_i)
            # start the generator
            next(i) 
        return inner
    return f